# Plots features vs errors
THE IDEA IS SEE IF THE VALUES OF SOME FEATURES HAS HIGHER ERRORS

THIS HAS THE SAME IDEA OF PARTIAL DEPENDECE PLOT BUT INSTEAD OF PLOT y_predicted vs feature IN THIS NOTEBOOKS ARE PLOTEED errors vs feature


---------
Plots of predictions of the models with any kind of transformation

**IMPORTANT**: The list of models to evaluate is the same, but each model could have its own feature eng, but the Input (the data_X) and the Output (the prediction) follow the same structure, so it is necesary only one notebook to evaluate the differents notebooks of training (if it is not logic for you thinking in the kaggle competitions).

In this notebook, there are a parameter "folder_models" and in this folder are located the pkl of each model


In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## RUN

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

### 0. Global params

In [ ]:
# define folder where the models were saved. There are the same models accepted by gurobi but the feature eng changed

# list of folder with models = ['basic', 'scaler', 'poly_2', 'poly_3']
folder_models = 'poly_3'

### 1. Load data

In [ ]:
### DEFINE LIST FEARTURES - TARGET (order data to have the same order in the features always)
list_features = ['AveOccup', 'Latitude', 'Population', 'AveBedrms', 'HouseAge', 'Longitude', 'AveRooms', 'MedInc']
target = 'Price'

In [ ]:
### LOAD DATA
X_train = pd.read_pickle('artifacts/data/X_train.pkl')
X_test = pd.read_pickle('artifacts/data/X_test.pkl')
y_train = pd.read_pickle('artifacts/data/y_train.pkl')
y_test = pd.read_pickle('artifacts/data/y_test.pkl')

In [ ]:
print('shape data')
print('\n\n TRAIN')
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('\n\n TEST')
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

In [ ]:
# sample - run fast
X_train = X_train[0:100]
y_train = y_train[0:100]

### 2. Load Models
Load all the models in a dictory

In [ ]:
## define list of models - list to have always the same order.
#### In this example, the strings in the list are the same with the models were saved
list_models_names = [
    "lr",
    "ridge",
    "lasso",
    
    "tree_simple",
    "tree_default",
    
    "rf_simple",
    "rf_default",

    "gb_simple",
    "gb_default",

    "xgb_simple",
    "xgb_default",

    "mlp_simple",
    "mlp_default"
]

In [ ]:
# define path to folder models
path_folder_models = f'artifacts/models/{folder_models}/'

In [ ]:
### load models
dict_models = {}
for model_name in list_models_names:
    print(f'loading model: {model_name}')
    path_model = path_folder_models + f'{model_name}.pkl'
    with open(path_model, 'rb') as artifact:
        dict_models[model_name] = pickle.load(artifact)

### 3. Define function to plot feature vs error (subplots each feature)

In [ ]:
def plot_errors_vs_one_feature(X, y, y_pred, title_plot, list_features, abs_error):
    """
    Plot errors vs features.

    Args:
        X (dataframe): dataframe with X true values - features
        y (dataframe): dataframe with y-true values 
        y_pred (dataframe): dataframe with y-pred values
        title_plot (string): tittle in the plot
        list_features (list): list of features that will plot againts the errrors. The features needs to be present in the data
        abs_error (boolean): True: Plot the absolute value of the errors abs(y_true - y_pred). False: Plot y_true - y_pred
    
    Return
        fig (figure matplolib): figure to show, download, etc
    """

    # calculate error
    errors = y - y_pred # error
    errors_abs = np.abs(errors) # errors in abs value
    if abs_error:
        errors_to_plot = errors_abs
    else:
        errors_to_plot = errors

    # for in features
    for feature in list_features:

        fig = plt.Figure()
        plt.scatter(X[feature],  # feature
                   errors_to_plot,  # errors y_true - y_pred
                    alpha = 0.5
                   )
        
        # Add names to axis
        plt.xlabel(f'Feature: {feature}')
        plt.ylabel('Errors y')

        plt.title(title_plot)
        #plt.legend()
        plt.show()

In [ ]:
## example

# params
model_example = dict_models['lr']
y_example_true = y_train
y_example_pred = model_example.predict(X_train)
y_example_pred = pd.DataFrame(y_example_pred, index = y_example_true.index, columns = y_example_true.columns)

# plot
plot_errors_vs_one_feature(X = X_train,
                               y = y_example_true, 
                        y_pred = y_example_pred, 
                        title_plot = 'lr train',
                        list_features = list_features,
                        abs_error = True)

### 4. Define function to plot features (subplots) vs errors (One subplot for model)
Have a for to plot all the models and save each model individually

In [ ]:
def plot_errors_vs_all_features(model, X, y, abs_error):
    """
    Plot errors vs features - all features in the data X

    Args:
        model (sklearn model): model sklearn
        X (dataframe): dataframe with features
        y (dataframe): dataframe with target (y_true)
        abs_error (boolean): True: Plot the absolute value of the errors abs(y_true - y_pred). False: Plot y_true - y_pred
    
    Return
        fig (figure matplolib): figure to show, download, etc
    """
    
    # predict y_pred
    y_pred = model.predict(X)
    y_pred = pd.DataFrame(y_pred, index = y.index, columns = y.columns)
    
    # calculate error
    errors = y - y_pred # error
    errors_abs = np.abs(errors) # errors in abs value
    if abs_error:
        errors_to_plot = errors_abs
    else:
        errors_to_plot = errors

    # create subplots
    number_features = len(X.columns.tolist())
    fig, ax = plt.subplots(number_features, 1, figsize = ((7, 50)) , dpi = 300)
    
    for index, feature in enumerate(X.columns.tolist()):
    
        # plot scatter feature vs error
        ax[index].scatter(X[feature],  # feature
                           errors_to_plot,  # errors
                            alpha = 0.5
                           )
    
        # Add names to axis
        ax[index].set_xlabel(f'Feature: {feature}')
        ax[index].set_ylabel('Errors y')
    
        # layout
        ax[index].set_title(f'Plot Errors y - Feature: {feature}')
    
    
    # Adjust vertical spacing between subplots
    fig.subplots_adjust(wspace=0.5)
    
    # Automatically adjust layout to avoid overlapping elements
    plt.tight_layout()

    
    return fig

In [ ]:
# PLOT FEATURES VS ERRORS - TRAIN DATA
for index, model_name in enumerate(dict_models):
    print(index)
    print(model_name)

    fig_true_pred_models_train = plot_errors_vs_all_features(model = dict_models[model_name],
                                                             X = X_train,
                                                             y = y_train,
                                                             abs_error = True
                                                            )
    
    # save plot
    fig_true_pred_models_train.savefig(f'artifacts/plots_predictions_true_pred/{folder_models}/errors_features/plot_errors_features_{model_name}_train.png', 
                                       dpi = 300)

In [ ]:
# PLOT FEATURES VS ERRORS - TEST DATA
for index, model_name in enumerate(dict_models):
    print(index)
    print(model_name)

    fig_true_pred_models_test = plot_errors_vs_all_features(model = dict_models[model_name],
                                                             X = X_test,
                                                             y = y_test,
                                                             abs_error = True
                                                            )
    
    # save plot
    fig_true_pred_models_test.savefig(f'artifacts/plots_predictions_true_pred/{folder_models}/errors_features/plot_errors_features_{model_name}_test.png', 
                                       dpi = 300)

### 5. Define function to plot features (subplots) vs errors (One subplot for model) (all models in one plot)
Have a for to plot all the models and save each model individually

In [ ]:
def plot_errors_vs_feature_all_models(dict_models, X, y, feature, abs_error):
    """
    Plot errors vs value of one features - plot for all models into one plot - plotly

    Args:
        dict_models(dictionary): python dictionary where each element are differents models
        X (dataframe): dataframe with features
        y (dataframe): dataframe with target (y_true)
        feature (string): feature to plot
        abs_error (boolean): True: Plot the absolute value of the errors abs(y_true - y_pred). False: Plot y_true - y_pred
    
    Return
        fig (figure plotly): fig of plotly with the plot generated 
    """
    
    # generate dataframe with y_true, y_pred, model
    df_error = pd.DataFrame()
    
    for model_name, model in dict_models.items():
        # calculate y_pred
        y_pred = model.predict(X)
        y_pred = pd.DataFrame(y_pred, index = y.index, columns = y.columns)
    
        # calculate error
        errors = y - y_pred # error
        errors_abs = np.abs(errors) # errors in abs value
        if abs_error:
            errors_to_plot = errors_abs
        else:
            errors_to_plot = errors
    
        # save dataframe
        df_error_aux = pd.DataFrame({'error': errors_to_plot.values.flatten(), 'Model': model_name, f'{feature}': X[feature]})
        df_error = pd.concat([df_error, df_error_aux], ignore_index=True)
    
    
    # plot scatter plot - y_true vs y_pred
    fig = px.scatter(df_error, x = f'{feature}', y = 'error', color='Model', title='Errors vs Features by Model', opacity = 0.6)

    return fig

In [ ]:
# PLOT DATA TRAIN
fig_true_pred_all_models_train = plot_errors_vs_feature_all_models(dict_models = dict_models, 
                                                                   X = X_train, 
                                                                   y = y_train,
                                                                   feature = 'AveOccup',
                                                                   abs_error = True
                                                                  )

# show
#fig_true_pred_all_models_train

# save
fig_true_pred_all_models_train.write_html(f'artifacts/plots_predictions_true_pred/{folder_models}/errors_features/plot_errors_features_all_models_train.html')

# TODO:
- Plots with distribution of 2 features (axis x - axis y) and the error colored (like heatmap)